import pandas as pd
import re
from pandas import Timestamp
import statsmodels
from operator import itemgetter
import string
import numpy as np
from collections import Counter
from sklearn.metrics import accuracy_score,classification_report
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords as sw
from nltk.corpus import wordnet as wn
from nltk import wordpunct_tokenize
from nltk import WordNetLemmatizer
from nltk import sent_tokenize
from nltk import pos_tag
from nltk import bigrams
from nltk import word_tokenize
from nltk.sentiment.util import mark_negation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.base import TransformerMixin
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

df_A = pd.read_csv('a_file.csv', encoding='latin1')

print(list(df_A))
print(df_A.shape)

df_A['postcode'] = df_A['outcode'].map(str) + ' ' + df_A['incode'].map(str) 
df_A.shape

### Have a problem- postcode is sometimes in displayable address, so preserves the bit in displayabe address (and looks messy). Cut the postcode out of the displayable address:

df_A['displayable_address'] = df_A['displayable_address'].str.lower()
df_A['incode'] = df_A['incode'].str.lower()
df_A['outcode'] = df_A['outcode'].str.lower()

%timeit df_A['displayable_address'] = [''.join(str(e).split(k)) for e, k in zip(df_A.displayable_address.astype('str'), df_A.outcode.astype('str'))]

%timeit df_A['displayable_address'] = [''.join(str(e).split(k)) for e, k in zip(df_A.displayable_address.astype('str'), df_A.incode.astype('str'))]

df_A.drop(['outcode', 'incode'], axis=1, inplace=True)
print(list(df_A))
print(df_A.shape)

### Concatenate and deduplicate address

df_A['address'] = df_A['street_name'].map(str) + ' ' + df_A['displayable_address'].map(str) + ' ' + df_A['post_town'].map(str) + ' ' + df_A['county'].map(str) + ' ' + df_A['country'].map(str)  + ' ' + df_A['postcode'].map(str)

df_A['address'] = df_A['address'].apply(lambda x : x.encode('ascii', errors='ignore').decode('latin1'))

df_A['address'] = df_A['address'].str.lower()
df_A['address'] = df_A['address'].str.replace(',', '')
df_A['address'] = df_A['address'].str.replace('shrie', 'shire')
df_A['address'] = df_A['address'].apply(lambda x: re.sub(' +', ' ',x))
df_A['address'] = df_A['address'].apply(lambda x: ' '.join(pd.unique(x.split())))
df_A['address'].head()

df_A = df_A.drop(['street_name','displayable_address','post_town', 'county', 'country'], axis=1)

df_A['address'] = df_A['address'].str.replace('\n',' ')
df_A['address'] = df_A['address'].str.replace('&amp;pound;','£')
df_A['address'] = df_A['address'].str.replace('&amp;','&')
df_A['address'] = df_A['address'].str.replace('&amp;amp','&')
df_A['address'] = df_A['address'].str.replace('&amp;quot','"')
df_A['address'] = df_A['address'].str.replace('&amp;nbsp',' ')
df_A['address'] = df_A['address'].str.replace('&amp;#39',"'")
df_A['address'] = df_A['address'].str.replace('&#39',"'")
df_A['address'] = df_A['address'].str.replace('&#x2019',"'")
df_A['address'] = df_A['address'].str.replace('&#x2018',"'")
df_A['address'] = df_A['address'].str.replace('&#x2013',"-")
df_A['address'] = df_A['address'].str.replace('.&#13;&#10;'," ")
df_A['address'] = df_A['address'].str.replace('&#8217;',"'")
df_A['address'] = df_A['address'].str.replace('&nbsp;',' ')
df_A['address'] = df_A['address'].str.replace('&quot','"')
df_A['address'] = df_A['address'].str.replace('&#231',"'")

df_A['description'] = df_A['description'].str.replace('\n',' ')
df_A['description'] = df_A['description'].str.replace('&amp;pound;','£')
df_A['description'] = df_A['description'].str.replace('&amp;','&')
df_A['description'] = df_A['description'].str.replace('&amp;amp','&')
df_A['description'] = df_A['description'].str.replace('&amp;quot','"')
df_A['description'] = df_A['description'].str.replace('&amp;nbsp',' ')
df_A['description'] = df_A['description'].str.replace('&amp;#39',"'")
df_A['description'] = df_A['description'].str.replace('&#39',"'")
df_A['description'] = df_A['description'].str.replace('&#x2019',"'")
df_A['description'] = df_A['description'].str.replace('&#x2018',"'")
df_A['description'] = df_A['description'].str.replace('&#x2013',"-")
df_A['description'] = df_A['description'].str.replace('.&#13;&#10;'," ")
df_A['description'] = df_A['description'].str.replace('&#8217;',"'")
df_A['description'] = df_A['description'].str.replace('&nbsp;',' ')
df_A['description'] = df_A['description'].str.replace('&quot','"')
df_A['description'] = df_A['description'].str.replace('&#231',"'")

nspl = pd.read_csv('NSPL_MAY_2017_UK.csv')

nspl2 = pd.read_csv('NSPL_MAY_2017_UK.csv', usecols=['pcds', 'laua', 'msoa11', 'lat', 'long', 'oseast1m', 'osnrth1m'])

df_geog = pd.merge(df_A, nspl2, left_on='postcode', right_on='pcds')
df_geog.drop(['pcds'], axis = 1, inplace= True)


df_geog.to_csv('ZOOPLA_A.csv')

## Find gated communities

df_geog['description']= df_geog['description'].str.lower()

df_geog['gatedcom'] = df_geog['description'].str.contains("gated community|gated property|gated development|gated cul-de-sac|gated mews|gated complex")
df_geog['gated'] = df_geog['description'].str.contains("gated")


gated = df_geog[df_geog['gatedcom'] == True].copy()
#gated.drop('gatedcom', axis=1, inplace=True)

gated['description'].iloc[1]

gated.shape

gated.to_csv('gatedAsample.csv')

df_geog['gated'] = df_geog['description'].str.contains("gated")

gate = df_geog[df_geog['gated']==True].copy()
gate.drop('gated', axis=1, inplace=True)

gate.shape

gate.to_csv('gated_A.csv')

import numpy as np
np.random.seed(42)
sample = gate.loc[np.random.choice(gate.index, 500, replace = False)]

sample.shape

sample.to_csv('gated_sample_A.csv')

dfnotgated = df_geog[df_geog['description'].str.contains("gated")==False]

np.random.seed(42)
notgated= dfnotgated.loc[np.random.choice(dfnotgated.index, 500, replace=False)]
notgated.shape

print(list(notgated))
notgated.drop('caravan',axis=1, inplace=True)


### Caravans

df_geog['caravan'] = df_geog['description'].str.contains("caravan|park home|mobile home|static home|static caravan")

caravan = df_geog[df_geog['caravan']==True].copy()


caravan.shape

caravan.to_csv('caravan_A.csv')

not_caravan = df_geog[df_geog.description.str.contains("caravan|park home|mobile home|static home|static caravan")==False]

not_caravan.shape

not_caravan.to_csv('not_caravan_A.csv')